[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/instant-ngp-colab/blob/main/instant_ngp.ipynb)

In [ ]:
# !git lfs install
# !git clone https://huggingface.co/camenduru/instant-ngp

!wget https://huggingface.co/camenduru/instant-ngp/resolve/main/instant-ngp.zip
!unzip /content/instant-ngp.zip -d instant-ngp

!pip install commentjson
%cd /content/instant-ngp
import os

In [ ]:
scene_path = "/content/instant-ngp/data/nerf/fox" #@param {type:"string"}
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)

train_steps = 2000  #@param {type:"integer"}
snapshot_path = os.path.join(scene_path, f"{train_steps}.msgpack")
!python ./scripts/run.py --scene {scene_path} --mode nerf --n_steps {train_steps} --save_snapshot {snapshot_path}